# Этика ИИ
## Итоговый проект. Николаев А.М.

### Загрузка необходимых библиотек и датасета

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
file_path = r'C:\Users\User\Desktop\etika_ii\data.csv'
data = pd.read_csv(file_path)

data.sample(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
21695,5204917,0.904762,flagged for your antisemite stupidity and hate...,0.023810,0.142857,0.166667,0.857143,0.0,0.0,0.0,...,331259,rejected,0,0,0,0,0,0.0,5,42
44235,6276191,1.000000,But did he have any attack chickens?\nSaw a po...,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.0,...,396013,approved,0,0,0,2,0,0.0,4,4
5080,491603,1.000000,"Christopher, history reminds us that you can't...",0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.0,...,146784,approved,0,0,0,1,0,0.0,10,4
45481,5598527,0.000000,No need to have over paid technicians like Kyl...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,355689,approved,1,0,0,4,0,0.0,0,4
84883,5783856,0.000000,You have a point. Low time pilots and Alaskan...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,367051,approved,0,0,0,1,0,0.0,0,4
66219,1007323,0.000000,"""Many Canadians too cash-strapped to raise chi...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,168445,approved,0,0,0,3,2,0.0,0,4
15131,965886,0.774194,You mean the Liberals should do what the Conse...,0.048387,0.290323,0.016129,0.725806,0.0,NaN,NaN,...,166795,rejected,0,0,0,0,0,0.0,0,62
23685,5304574,0.843750,Another commenter with zero credibility. \nYou...,0.046875,0.140625,0.015625,0.859375,0.0,NaN,NaN,...,337266,approved,0,0,0,2,1,0.0,0,64
18292,5026061,0.800000,ignorance never ends from you left wingers,0.000000,0.000000,0.000000,0.800000,0.0,NaN,NaN,...,320575,rejected,0,0,0,3,3,0.0,0,5
52948,5929873,0.000000,"Pray all you want Skeletor, prayer and a nicke...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,376212,approved,0,0,0,0,0,0.0,0,4


Задача
В конце 2017 года платформа Civil Comments закрылась и опубликовала около 2 миллионов комментариев из социальных сетей, чтобы специалисты по данным со всего мира могли работать вместе над исследованием способов смягчения предвзятости в текстовых данных.

Таблица с комментариями также прилагается: data.csv. Мы будем работать с тысячами комментариев, где каждый комментарий помечен как «токсичный» или «нетоксичный».

В таблице data.csv в колонке comment_text написаны сами комментарии, с которыми нам предстоит работать. В колонке target стоят вероятности того, что комментарий токсичен. Давайте сделаем предпосылку, что комментарий считается токсичным, если вероятность выше 0.7.

Выполните код ниже, чтобы отделить колонки и преобразовать вероятности в бинарные величины, где 1 — токсичный комментарий, а 0 — нетоксичный комментарий.

In [3]:
data['binary_target'] = (data['target'] > 0.7).astype(int)

comments = data['comment_text']
target = data['binary_target']

print(comments.sample(10))
target.sample(10)

21886    Women in their 30's without children are usual...
56873    2/2\nI suspect you've never read a word Bernar...
56515    AKHunter, I was just looking at the Wood-Tickc...
26096                    Darn, all those onions again ....
37812    Wow Exedus, you are working this thread on OT....
29497    What an idiot. Your saying retired people are ...
60634    Golfing? Likely! In day 202 of his presidency,...
9463     Ontario elected a School Teacher,so why is any...
38441    What the f are you talking about?  Is this sar...
52298    My wife found a home with a fully assumable 16...
Name: comment_text, dtype: object


74524    0
53719    0
505      1
6412     1
33358    1
54937    0
86877    0
63543    0
40383    1
42607    1
Name: binary_target, dtype: int64

#### Задание 1

Теперь разделим наши данные на train и test. Пусть в тест у нас пойдет 30% данных. Для этого можете использовать библиотеку train_test_split из sklearn.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(comments, target, test_size=0.3, random_state=0)

print(len(X_train), len(X_test), len(y_train), len(y_test))

63631 27271 63631 27271


#### Задание 2

Как вы уже, наверное, догадались, в наших данных есть определенная проблема — это разные типы данных. Предсказывать мы хотим бинарную величину: 1 и 0. А на вход мы подаем текст, а не числа. Поэтому нам нужно как-то сконвертировать текст в число.

Давайте это сделаем!

Вам понадобится функция CountVectorizer: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Преобразуйте текст, который вы поделили на train и test, в числовой формат с помощью этой функции.

In [5]:
vectorizer = CountVectorizer()

X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

X_train_vectorized.shape, X_test_vectorized.shape

((63631, 58152), (27271, 58152))

#### Задание 3

Теперь в качестве модели, которая будет классифицировать нам комментарии на токсичные и нетоксичные, возьмем логистическую регрессию.

Импортируйте из библиотеки sklearn логистическую регрессию LogisticRegression с параметром max_iter=2000. Для оценки модели возьмите метрику accuracy и посчитайте ее.

In [6]:
classifier = LogisticRegression(max_iter=2000, random_state=0)
classifier.fit(X_train_vectorized, y_train)

y_pred = classifier.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)

accuracy

0.9287154853140699

#### Задание 4

Чтобы мы смогли протестировать разные комментарии, которые приходят в голову, пропишите ниже функцию, для которой на вход мы бы подавали наш комментарий, а на выход получали предсказание, насколько от 0 до 1 комментарий является токсичным.

In [7]:
def predict_toxicity(com, vectorizer, classifier):

    vectorized_comment = vectorizer.transform([com])
    probability = classifier.predict_proba(vectorized_comment)[0, 1]
    return probability

#### Задание 5

Поздравляю! Вы только что написали первый бейзлайн для определения токсичности комментариев! :)

А теперь посмотрим, насколько он этичен.

Попробуйте предсказать, токсичен ли комментарий «Apples are stupid». Потом предскажите, токсичен ли комментарий «I love apples».

In [8]:
com1 = "Apples are stupid"
com2 = "I love apples"

toxicity1 = predict_toxicity(com1, vectorizer, classifier)
toxicity2 = predict_toxicity(com2, vectorizer, classifier)

print(f"'{com1}' - токсичность: {toxicity1:.2f}")
print(f"'{com2}' - токсичность: {toxicity2:.2f}")

'Apples are stupid' - токсичность: 1.00
'I love apples' - токсичность: 0.10


#### Задание 6

Если ваш алгоритм работает корректно, то комментарий «I love apples» должен быть определен как нетоксичный, а «Apples are stupid» — как токсичный.

А теперь перейдем к пониманию того, как модель принимает решения: модель присваивает каждому из примерно 58 000 слов коэффициент, причем более высокие коэффициенты обозначают слова, которые модель считает более токсичными. Выведите десять слов, которые считаются наиболее токсичными, а также их коэффициенты.

Hint: в этом вам поможет атрибут vectorizer.vocabulary_.keys() и classifier.coef_

In [9]:
word_coefs = list(zip(vectorizer.get_feature_names_out(), classifier.coef_[0]))
sorted_words = sorted(word_coefs, key=lambda x: x[1], reverse=True)

top_toxic_words = sorted_words[:10]
for word, coef in top_toxic_words:
    print(f"{word}: {coef:.4f}")

stupid: 9.4708
idiot: 8.6652
idiots: 8.6195
stupidity: 7.5686
idiotic: 6.7927
crap: 6.5523
dumb: 6.5368
pathetic: 6.4536
morons: 6.3361
moron: 6.3253


#### Задание 7

Взгляните на самые токсичные слова из задания 6. Вызывают ли у вас удивление какие-нибудь из них? Есть ли слова, которых, кажется, не должно быть в списке?

В целом удивления не вызывает, скорее разочаровывает, что почти все слова связаны с оценкой интеллектуальных способностей, я ожидал более ярких ругательств UwU

#### Задание 8

Давайте попробуем протестировать модель на ее предвзятость, например, к религии.

Примечание: DISCLAIMER мы обсуждали это на лекциях и семинарах, но повторим еще раз: наша задача — отследить предвзятость алгоритма, а не обидеть или оскорбить представителей определенной национальности, религии, политических кругов и т.д.
Давайте посмотрим, как ваш алгоритм классифицирует следующие комментарии:

- "I have a christian friend"
- "I have a muslim friend"
- "I have a white friend"
- "I have a black friend"

Что думаете о получившихся результатах? Есть ли у модели bias? Этичен ли он?

In [10]:
comments = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend"
          ]

for comment in comments:
    toxicity = predict_toxicity(comment, vectorizer, classifier)
    print(f"'{comment}' - токсичность: {toxicity:.2f}")

'I have a christian friend' - токсичность: 0.12
'I have a muslim friend' - токсичность: 0.45
'I have a white friend' - токсичность: 0.33
'I have a black friend' - токсичность: 0.51


Модель получилась предвзято "относится" к мусульманам, афроамериканцам и европейцам. А вот с христинами "всё в порядке".

#### Задание 9

Немного теории: в этике ИИ принято писать такой алгоритм, который будет соответствовать 4 критериям этики:

1. Демографический паритет/статистический паритет. Демографический паритет говорит о том, что модель является справедливой, если состав людей, выбранных с помощью модели, соответствует проценту членства в группе претендентов.

Некоммерческая организация организует международную конференцию, на участие в которой записались 20 000 человек. Организаторы пишут модель машинного обучения, чтобы отобрать 100 участников, которые потенциально могли бы выступить с интересными докладами на конференции. Поскольку 50% участников будут женщинами (10 000 из 20 000), они разрабатывают модель так, чтобы 50% выбранных кандидатов были женщинами.

2. Равные возможности. Справедливость равных возможностей гарантирует, что доля людей, которые должны быть выбраны с помощью модели («позитивы»), и которые правильно выбраны с помощью модели, одинакова для каждой группы. Мы называем эту пропорцию истинно положительным показателем (TPR) или чувствительностью модели.

Врач использует инструмент для выявления пациентов, нуждающихся в дополнительной помощи, которые могут подвергаться риску развития серьезных заболеваний. (Этот инструмент используется только в качестве дополнения к врачебной практике — второго мнения.) Он предназначен для обеспечения высокого TPR, одинакового для каждой демографической группы.

3. Одинаковая точность. В качестве альтернативы мы могли бы проверить, имеет ли модель одинаковую точность для каждой группы. То есть процент правильных классификаций (люди, которым следует отказать, которым отказано, и люди, заявку которых следует одобрить, которых одобряют) должен быть одинаковым для каждой группы. Если модель точна на 98% для людей в одной группе, она должна быть точна на 98% для других групп.

Банк использует модель для одобрения людям кредита. Модель спроектирована так, чтобы быть одинаково точной для каждой демографической группы: таким образом банк избегает одобрения людей, которым должно быть отказано (что нанесло бы финансовый ущерб как заявителю, так и банку), а также избегает отклонения людей, которые должны быть одобрены (что могло бы стать упущенной возможностью для заявителя и снизить доходы банка).

4. Группа по незнанию или «справедливость через неосведомленность». Группа не осознает справедливости и удаляет всю информацию о членстве в ней из набора данных. Например, мы можем удалить гендерные данные, чтобы попытаться сделать модель справедливой для разных гендерных групп. Аналогичным образом мы можем удалить информацию о расе или возрасте.

Одна из трудностей применения этого подхода на практике заключается в том, что нужно быть осторожным при идентификации и удалении прокси-серверов для данных о членстве в группах. Например, в городах, где существует расовая сегрегация, почтовый индекс является сильным показателем расы. То есть когда данные о расовой принадлежности удаляются, данные почтового индекса также должны быть удалены, иначе приложение ML все равно сможет определить расовую принадлежность человека на основе данных. Кроме того, группа, не осознающая справедливости, вряд ли станет хорошим решением проблемы исторической предвзятости.

Вы заметили, что комментарии, относящиеся к исламу, с большей вероятностью будут токсичными, чем комментарии, относящиеся к другим религиям, поскольку онлайн-сообщество исламофобно. Какой тип предвзятости это может внести в вашу модель?

У модели статистическая или системная предвзятость по отношению к афроамериканцам, мусульманам и европейцам, которая, судя по всему возникает из-за использования данных слов часто в негативном контексте.

#### Задание 10

Подумайте о том, как можно улучшить алгоритм, чтобы сделать его более этичным. Напишите 1–2 идеи.

- Можно заменить слова относящиеся к рассам на название вымышленой инопланетной расы, а религию на пастофарианство, чтобы избежать упоминаний о реальных религиях и рассах.
- Можно попробовать использовать контекстные модели, такие как BERT, которые лучше понимают контекст, а не ориентируются на токсичность отдельного слова.
- Можно попробовать сбалансировать тренировочный набор для всех рассматриваемых групп.